In [106]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import json
from urllib.request import urlopen

from tqdm import tqdm_notebook
import unidecode

In [71]:
df_debut = pd.read_csv("../data/debut_album_1118-utf8.csv")
df_debut.head()

,artist,album,genre,single_count,release_date
0,2Cellos,2Cellos,"Cello rock, classical crossover",3,"June 10, 2011"
1,Deep Dark Robot,8 Songs About a Girl,"Post-grunge, punk blues",0,"MarchÂ 22,Â 2011Â (2011-03-22)"
2,The Young Professionals,9am to 5pm 5pm to Whenever\r(09:00 to 17:00 17...,Electronic,0,"September 12, 2011 (Israel)\rJune 18, 2012 (in..."
3,Flash Bang Grenada,10 Haters,Alternative hip hop,0,"AugustÂ 23,Â 2011Â (2011-08-23)"
4,Her Majesty & the Wolves,111,"Dance, pop",3,"July 11, 2011 (UK)"


In [74]:
billboard_df = pd.DataFrame(columns = ['artist', 'album', 'billboard_frequency'])

for i in tqdm_notebook(range(len(df_debut['artist']))):
    
    artist = df_debut['artist'][i]
    album = df_debut['album'][i]

    artist_ = artist.replace(" ", "%20")
    album_ = album.replace(" ", "%20")

    keyword = artist_ + "%20" + album_
    
    url = "https://www.billboard.com/search/" + keyword

    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    try:
        article_count = int(str(soup.select("#main > div > div > div.section.article-section > div.section-header > div.section-header__count")[0].get_text())[10:-1])
    except IndexError:
        article_count = 0
    
    data = {
    'artist' : artist,
    'album' : album,
    'billboard_frequency' : article_count,
    }
    
    billboard_df.loc[len(billboard_df)] = data

HBox(children=(IntProgress(value=0, max=1951), HTML(value='')))

In [104]:
print(billboard_df.shape)
billboard_df.head()

(1951, 3)


,artist,album,billboard_frequency
0,2Cellos,2Cellos,0
1,Deep Dark Robot,8 Songs About a Girl,0
2,The Young Professionals,9am to 5pm 5pm to Whenever\r(09:00 to 17:00 17...,0
3,Flash Bang Grenada,10 Haters,0
4,Her Majesty & the Wolves,111,0


In [109]:
billboard_df['artist'] = billboard_df['artist'].apply(lambda x: unidecode.unidecode(x))
billboard_df['album'] = billboard_df['album'].apply(lambda x: unidecode.unidecode(x))
billboard_df['artist'] = billboard_df['artist'].apply(lambda x : x.replace(",", ""))
billboard_df['album'] = billboard_df['album'].apply(lambda x : x.replace(",", ""))

In [115]:
billboard_df.loc[2, 'album'] = '9am to 5pm 5pm to Whenever'
billboard_df.loc[529, 'album'] = 'If You Love Me'
billboard_df.loc[1143, 'album'] = 'Gover Tela Body Language'
billboard_df.loc[1411, 'album'] = 'Fresh Filter Volumn 1'

In [116]:
billboard_df.shape

(1951, 3)

In [118]:
billboard_df.to_csv("../data/buzz_billboard.csv", index=False, encoding="UTF-8")

In [119]:
df = pd.read_csv("../data/buzz_billboard-utf8.csv")
df.shape

(1951, 3)

### add to mysql database

In [120]:
import sqlalchemy, pickle
from sqlalchemy import create_engine

In [121]:
pw = pickle.load(open("mysql_pw.pickle", "rb"))

In [122]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/project_rookie")

In [123]:
df.to_sql(name="buzz_billboard", con=engine, if_exists='replace')